## Implementation of a stacked Classifier

## Evaluating the Model

In [1]:
from pipeline import pipeline
from chris import ldData, mlpClassification, makeTrainValSet, balanceStupid, NO_DISPLAY_savePred
from olin import stackedClassification, ecgExtractOlin
from operator import itemgetter
import matplotlib.pyplot as plt
import numpy as np
from heinrich import inv, crop, ecgExtract, rfClassification
from anova import anova

In [ ]:

hyper = { 
    
    "inv_threshold": 0.6, 
    "crop_location": 300,
    "mlpClassification_epochs": 200,
    "mlpClassification_useValidationSet": True,
    "mlpClassification_makePrediction": False,
    "makeTrainValSet_valPercent": 0.1,
    "rfClassification_depth": 3,
    "rfClassification_useValidationSet": True,
    "rfClassification_makePrediction": False,
    "anova_percentage": 0.7,
    "stackedClf_useValidationSet": True,
    "stackedClf_makePrediction": False,
    "used_cached_stats": True
    
}

data = pipeline([ ldData, crop, inv, ecgExtractOlin, makeTrainValSet, rfClassification ], hyper, save_states_to_cache=True)
print( "train losses" )
data["train_losses"]
print( "val losses" )
data["val_losses"]

  0%|          | 2/5116 [00:00<04:18, 19.79it/s]

[Pipeline] Saved state found: ./cache/ldData()_crop(300)_inv(0.6), starting from function: ecgExtractOlin
[Pipeline] executing: ecgExtractOlin()
extracting from X_train


 50%|████▉     | 2550/5116 [01:39<01:37, 26.38it/s]

In [ ]:
data["train_losses"]

In [ ]:
data["val_losses"]

In [ ]:
from olin_utils import confMat
_, misclass = confMat( data[ "y_val_hat" ], np.transpose( data[ "y_val" ])[ 0 ], visualize = True )

In [3]:
import numpy as np
def periodicity(peak_mask):
    '''
    Given a peak mask, this function creates some parameters from the difference of the length of the periods.
    
    peak_mask (np.ndarray or list): containing the indices of the peaks. E.g. [10, 210, 410, 614]    
    '''
    
    # compute the lengths of each period
    '''
    lengths_of_periods = []
    for i in range(len(peak_mask)-1):
        start_idx = peak_mask[i]
        end_idx = peak_mask[i+1]
        lengths_of_periods.append(end_idx - start_idx)
    '''
    lengths_of_periods = np.diff(peak_mask)
    print(lengths_of_periods)
    fs = [ np.min, np.max, np.mean, np.median, np.std ]
    return np.array([ f( lengths_of_periods ) for f in fs ])
        
    
    

In [4]:
periodicity([1, 10, 21, 32, 44])

[ 9 11 11 12]


array([ 9.        , 12.        , 10.75      , 11.        ,  1.08972474])